<a href="https://colab.research.google.com/github/mgnarag/binarization-autoencoder/blob/main/Traditional_binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/6 Traditional Binarization/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'107.1 syllabus 1SAY1920 FINAL.gdoc'
'1C-01 Narag et al (1).pdf'
'1C-01 Narag et al.pdf'
'1FA971504AC646559D05A8ED6341AFB3[1025340].png'
'1H-04 Narag et al.pdf'
 20220802_182749.heic
 20220802_182831.heic
 20220802_183259.heic
 20220803_125819.heic
 20220803_125842.heic
 20220803_144229.heic
 20220803_144422.heic
 20220803_160724.heic
 20220803_160751.heic
 20220803_165240.heic
 20220803_165318.heic
 20220803_174701.heic
 20220803_174706.heic
 20220804_175217.heic
 20220804_175443.heic
 20220804_175616.heic
 20220804_181917.heic
 20220804_182007.heic
 20220804_182117.heic
 2S22-23_LE3-Regular-Set-A-FINAL-KEY.pdf
'A4_Physics 71 TWHFU-3_gradesheet (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades.gdoc'
'Abstract - NARAG, Mark Jeremy.pdf'
'Annex 1 (copy of Journal).pdf'
'Annex 2 (Proof of employments).pdf'
'App Physics

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import matplotlib.pyplot as plt

import cv2

In [3]:
def metrics(gt,thresh1):
  out_bin = np.where(thresh1 == 255, 1, thresh1)
  gt_bin = np.where(gt == 255, 1, gt)
  gt_bin = gt_bin^1
  out_bin = out_bin^1

  tp = np.sum(out_bin & gt_bin)
  fp = np.sum(out_bin & ~gt_bin)
  fn = np.sum(~out_bin & gt_bin)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_score = (2*precision*recall)/(precision + recall)

  intersection = np.sum(np.logical_and(gt_bin, out_bin))
  union = np.sum(np.logical_or(gt_bin, out_bin))
  iou = intersection / union

  psnr = cv2.PSNR(gt, thresh1)
  return f1_score, iou, psnr


def simple_thresh(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  for i in range(0,255):
    ret,thresh1 = cv2.threshold(test,i,255,cv2.THRESH_BINARY)
    M = metrics(gt,thresh1)
    F1.append(M[0])
    IOU.append(M[1])
    PSNR.append(M[2])
  index = np.nanargmax(F1)
  return F1[index], IOU[index], PSNR[index], index

def otsu_thresh(gt,test):
  ret,thresh1 = cv2.threshold(test,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  M = metrics(gt,thresh1)
  return M[0], M[1], M[2]

In [4]:
def adaptive_thresh_gaussian(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 256, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    for i in range(0,100):
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,k,i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)

  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

def adaptive_thresh_mean(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 256, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    for i in range(0,100):
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,k,i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)
  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

In [5]:
gt = cv2.imread(file_path+ 'ground_truth_100dpi.png', cv2.IMREAD_GRAYSCALE)
test = cv2.imread(file_path+ 'nbs_100dpi.png', cv2.IMREAD_GRAYSCALE)
#for simple thresh:
simple = simple_thresh(gt,test)
print('simple thresh = ', simple)
otsu = otsu_thresh(gt,test)
print('simple thresh = ', otsu)

#saving:
simple_final = Image.fromarray(~output)
reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

<ipython-input-3-556818b699f7>:10: RuntimeWarning: invalid value encountered in ulong_scalars
  precision = tp / (tp + fp)


simple thresh =  (0.7625004190352133, 0.6161621634172582, 15.146347532884514, 109)
simple thresh =  (0.748887803731719, 0.5985776543186475, 14.448542747658957)


In [6]:
#for adaptive thresh:
gaussian = adaptive_thresh_gaussian(gt,test)
print('adaptive thresh = ', gaussian)
mean = adaptive_thresh_mean(gt,test)
print('adaptive thresh = ', mean)

<ipython-input-3-556818b699f7>:10: RuntimeWarning: invalid value encountered in ulong_scalars
  precision = tp / (tp + fp)


KeyboardInterrupt: ignored

In [ ]:
thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,65,50)
plt.imshow(thresh1, cmap = 'gray' )